In [44]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)
    
from utils.generate_dataset import generate_windows_dataset
from utils.features import decomp_and_features


data_dir_path = os.path.join("..", "data", "gridloss", "train.csv")
df = pd.read_csv(data_dir_path, index_col=0)
df.index = pd.to_datetime(df.index)

# Backfill missing data
df = df.bfill()

grid1_columns = ["grid1-load","grid1-loss","grid1-temp"]
window_size = 168 # 1 week
step_size = 24
num_ts = len(grid1_columns)
dataset_size = (df.shape[0] - window_size)//step_size + 1
force_calc_windows: bool = True

windows_data_paths = [os.path.join("..", "data", "gridloss", "windows", f"ts{i}_w{window_size}_s{step_size}_n{dataset_size}.csv") for i in range(num_ts)]
load_windows: bool = all([os.path.exists(path) for path in windows_data_paths])

if not load_windows or force_calc_windows:
    print("Generating windows dataset")
    data = generate_windows_dataset(df, window_size, step_size, grid1_columns)
else:
    # TODO: Add datetime index to the windows that are saved/loaded
    print("Loading windows dataset")
    data_transposed = [pd.read_csv(path).to_numpy() for path in windows_data_paths]
    data = np.asarray(data_transposed).transpose(1, 2, 0).tolist()
    data = [pd.DataFrame(data[i], columns=grid1_columns) for i in range(len(data))]

Generating windows dataset


100%|██████████| 724/724 [00:00<00:00, 938.80it/s]


# Decompositions

In [45]:
decomps, features = decomp_and_features(data, series_periodicity=24)

100%|██████████| 724/724 [00:14<00:00, 51.48it/s]


# Transformations

In [46]:
from utils.transformations import manipulate_trend_component, manipulate_seasonal_component

In [47]:
# Multiplicative constants
f = 1.3 # trend determination
g = 1.1 # trend slope
h = 0.5 # trend linearity
m = 0 # additional trend (not sure what to set this to)
k = 0.8 # seasonal determination

In [48]:
new_trends = [[manipulate_trend_component(decomp.trend, f, g, h, m) for decomp in mts_decomp] for mts_decomp in decomps]
new_seasonals = [[manipulate_seasonal_component(decomp.seasonal, k) for decomp in mts_decomp] for mts_decomp in decomps]

In [49]:
np.asarray(new_trends).shape, np.asarray(new_seasonals).shape

((724, 3, 168), (724, 3, 168))

In [50]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [75]:
i = 20

fig = make_subplots(rows=3, cols=2)
fig.add_trace(go.Line(x=decomps[i][0].trend.index, y=decomps[i][0].trend, name="grid1-load"), row=1, col=1)
fig.add_trace(go.Line(x=decomps[i][1].trend.index, y=decomps[i][1].trend, name="grid1-loss"), row=2, col=1)
fig.add_trace(go.Line(x=decomps[i][2].trend.index, y=decomps[i][2].trend, name="grid1-temp"), row=3, col=1)
fig.add_trace(go.Line(x=new_trends[i][0].index, y=new_trends[i][0], name="t-grid1-load"), row=1, col=2)
fig.add_trace(go.Line(x=new_trends[i][1].index, y=new_trends[i][1], name="t-grid1-loss"), row=2, col=2)
fig.add_trace(go.Line(x=new_trends[i][2].index, y=new_trends[i][2], name="t-grid1-temp"), row=3, col=2)
fig.update_layout(height=600, width=800, title_text="Trends before and after transformation")
fig.add_annotation(text=f"{f=}, {g=}, {h=}, {m=}", xref="paper", yref="paper", x=-0.07, y=1.1, showarrow=False)
fig.show()

In [ ]:
fig = make_subplots(rows=3, cols=2)
fig.add_trace(go.Line(x=decomps[i][0].seasonal.index, y=decomps[i][0].seasonal, name="grid1-load"), row=1, col=1)
fig.add_trace(go.Line(x=decomps[i][1].seasonal.index, y=decomps[i][1].seasonal, name="grid1-loss"), row=2, col=1)
fig.add_trace(go.Line(x=decomps[i][2].seasonal.index, y=decomps[i][2].seasonal, name="grid1-temp"), row=3, col=1)
fig.add_trace(go.Line(x=new_seasonals[i][0].index, y=new_seasonals[i][0], name="t-grid1-load"), row=1, col=2)
fig.add_trace(go.Line(x=new_seasonals[i][1].index, y=new_seasonals[i][1], name="t-grid1-loss"), row=2, col=2)
fig.add_trace(go.Line(x=new_seasonals[i][2].index, y=new_seasonals[i][2], name="t-grid1-temp"), row=3, col=2)
fig.update_layout(height=600, width=800, title_text="Season before and after transformation")
fig.add_annotation(text=f"{k=}", xref="paper", yref="paper", x=-0.07, y=1.1, showarrow=False)
fig.show()

In [ ]:
fig = make_subplots(rows=3, cols=2)
ts = [decomps[i][j].trend + decomps[i][j].seasonal + decomps[i][j].resid for j in range(3)]
new_ts = [new_trends[i][j] + new_seasonals[i][j] + decomps[i][j].resid for j in range(3)]
fig.add_trace(go.Line(x=decomps[i][0].trend.index, y=ts[0], name="grid1-load"), row=1, col=1)
fig.add_trace(go.Line(x=decomps[i][1].trend.index, y=ts[1], name="grid1-loss"), row=2, col=1)
fig.add_trace(go.Line(x=decomps[i][2].trend.index, y=ts[2], name="grid1-temp"), row=3, col=1)
fig.add_trace(go.Line(x=new_trends[i][0].index, y=new_ts[0], name="t-grid1-load"), row=1, col=2)
fig.add_trace(go.Line(x=new_trends[i][1].index, y=new_ts[1], name="t-grid1-loss"), row=2, col=2)
fig.add_trace(go.Line(x=new_trends[i][2].index, y=new_ts[2], name="t-grid1-temp"), row=3, col=2)
fig.update_layout(height=600, width=800, title_text="Time series before and after transformation")
fig.add_annotation(text=f"{f=}, {g=}, {h=}, {m=}, {k=}", xref="paper", yref="paper", x=-0.07, y=1.1, showarrow=False)
fig.show()